# Проект большие языковые модели

### Подготовка окружения

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast, LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments,TrainerCallback, AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW 
from datasets import load_dataset
import os
import gc
import torch
import warnings
from torch.utils.data import DataLoader
import regex as re
import numpy as np
import random
from pathlib import Path
from datasets import Dataset
import unicodedata
from collections import Counter
from tqdm.notebook import tqdm
from trl import SFTTrainer

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(42)
warnings.filterwarnings('ignore')

# освобождение памяти
def gc_collect():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize() 

gc_collect()

In [3]:
DATA_DIR = Path("RussianNovels/corpus")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 3000
CYRILLIC_RE = re.compile(r"^[\p{IsCyrillic}\s0-9.,!?—–:;\"'()«»…\-]+$")
BATCH_SIZE = 2 if DEVICE=='cpu' else 12
MAX_EPOCHS = 1
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0
SAVE_PATH = "best_model.pt"

TEST_PROMPTS = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
] 
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

QUESTIONS = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
]

print(f"{DEVICE=}")

DEVICE='cuda'


In [ ]:
!git clone https://github.com/JoannaBy/RussianNovels.git
!git clone https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru

## Pretrain

### Подготовка корпуска текстов

In [ ]:
def collect_texts(root: Path):
    texts = []
    for path in root.rglob("*"):
        if path.suffix in {".txt"}:
            try:
                texts.append(path.read_text(encoding="utf-8"))
            except Exception:
                pass
    return texts

raw_texts = collect_texts(DATA_DIR)
raw_texts = list(set(raw_texts))
print(f"Файлов в корпусе: {len(raw_texts)}")

In [ ]:
def filter_cyrillic_sentences(text: str):
    '''фильтруем предложения с некириллическими символами'''
    sentences = re.split(r"[.!?]+", text)
    return [
        s.strip()
        for s in sentences
        if s.strip() and CYRILLIC_RE.match(s.strip())
    ]

def normalize_punctuation(text: str) -> str:
    '''нормализуемы пунктуацию'''
    text = re.sub(r"[!?]{2,}", r"\1", text)
    text = re.sub(r"\.{3,}", "…", text)
    text = re.sub(r",,{2,}", ",", text)
    text = re.sub(r"-{2,}", "-", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[«»]', '"', text)
    return text.strip()    

clean_sentences = []

for text in raw_texts:
    sentences = filter_cyrillic_sentences(text)
    sentences = [normalize_punctuation(s) for s in sentences]
    clean_sentences.extend(sentences)

print(f"Предложений после очистки: {len(clean_sentences)}")

In [ ]:
print("Проверка состава оставшихся спец символов")
def is_letter_or_digit(ch: str) -> bool:
    cat = unicodedata.category(ch)
    return cat.startswith("L") or cat.startswith("N")

counter = Counter()

for sent in clean_sentences:
    for ch in sent:
        if not is_letter_or_digit(ch):
            counter[ch] += 1


non_alnum = counter.most_common()
print(f"symb | freg    | name")
for ch, freq in non_alnum[:30]:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"'{ch}'  | {freq:>7} | {name}")

In [ ]:
# разбиение на чанки
chunks = []
current = ""

for sent in clean_sentences:
    max_chars = 2000  # ~512 токенов
    if len(current) + len(sent) < max_chars:
        current += " " + sent
    else:
        chunks.append(current.strip())
        current = sent

if current:
    chunks.append(current.strip())

print(f"Чанков: {len(chunks)}")
print(f"чанк  0: {chunks[0]}")
print(f"чанк 10: {chunks[10]}")


### Обучение токенизатора

In [ ]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Нормализация текста
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
# разбиение по пробелам 
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"] 
    )
tokenizer.train_from_iterator(chunks, trainer)
# (BOS/EOS) 
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")), 
        ], 
        )
# Сохранение
tokenizer.save("bpe_ru_3k.json")

In [ ]:
encoded = tokenizer.encode("Пример текста для проверки токенизатора")
print(encoded.tokens)
print(encoded.ids)


### Сборка датасета

In [ ]:

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="bpe_ru_3k.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)

def tokenize(batch):
    return hf_tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def add_labels(batch):
    labels = []
    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        l = ids.copy()
        l = [
            token if m == 1 else -100
            for token, m in zip(l, mask)
        ]
        labels.append(l)

    batch["labels"] = labels
    return batch


dataset = Dataset.from_dict({"text": chunks})

dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
).map(add_labels, batched=True)

dataset.set_format(type="torch")

### Подготовка модели LlamaForCausalLM

In [ ]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    hidden_act="silu",
    tie_word_embeddings=True,
    attention_bias=False,
    rope_theta=10000.0,
    use_cache=False,     
)
config.vocab_size = max(hf_tokenizer.get_vocab().values()) + 1
config.pad_token_id = hf_tokenizer.pad_token_id
config.bos_token_id = hf_tokenizer.bos_token_id
config.eos_token_id = hf_tokenizer.eos_token_id

model = LlamaForCausalLM(config).to(DEVICE)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Обучаемых параметров: {num_params:,}")

In [ ]:
print("Проверим синхронизацию модели и текенизатора")
assert model.config.vocab_size == max(hf_tokenizer.get_vocab().values()) + 1
assert model.config.pad_token_id == hf_tokenizer.pad_token_id
assert model.config.eos_token_id == hf_tokenizer.eos_token_id
print("ок")

In [ ]:
print("Sanity-check начальной модели на инференсе")
model.eval()

prompt = "Привет"
inputs = hf_tokenizer(
    prompt,
    return_tensors="pt",
    return_token_type_ids=False
).to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=hf_tokenizer.eos_token_id,
    )

print(hf_tokenizer.decode(out[0], skip_special_tokens=True))


In [ ]:
# оптимизатор
def get_optim(model):
    return AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

optimizer = get_optim(model)

In [ ]:
print("Сheck сходимости модели при обучении на малом датасете (2 предожения)")
tiny_dataset = dataset.select(range(2))
tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=True)

model.train()

progress_bar = tqdm(
        range(201),
        desc=f"train check",
    )
best_loss = float("inf")

for step in progress_bar:
    batch = next(iter(tiny_loader))
    batch = {k: v.to(model.device) for k, v in batch.items()}

    loss = model(**batch).loss
    progress_bar.set_description(f"train check | loss: {loss:.2f}")
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step> 0 and step % 20 == 0:
        progress_bar.write(f"epoch: {step:>3} loss: {loss.item():.2f}")
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), SAVE_PATH)
            progress_bar.write(f"best model saved with {best_loss=:.2f}")

del tiny_dataset, tiny_loader, optimizer
gc_collect()

In [ ]:
print("Сheck инференса модели после обучения на малом датасете")
def check_inference(model, prompts: list):
    model.eval()
    for pr in prompts:
        inputs = hf_tokenizer(pr, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)

        out = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True, 
        eos_token_id=hf_tokenizer.eos_token_id,
    )
        print("input:  ",  pr)
        print("output: ", hf_tokenizer.decode(out[0], skip_special_tokens=True))
        print()


check_inference(model, TEST_PROMPTS[:2])

### Обучение модели LlamaForCausalLM

In [ ]:
# Callback для валидации процесса обучения
class PromptCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts, max_new_tokens=50):
        self.tokenizer = tokenizer
        self.prompts = prompts
        self.max_new_tokens = max_new_tokens

    def _one_answer(self, model, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                do_sample=True, 
                eos_token_id=hf_tokenizer.eos_token_id,
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        

    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        if model is None:
            return
        model.eval()
        print("="*20, f"on step {state.global_step}", "="*20)
        for prompt in self.prompts[:3]:
            decoded = self._one_answer(model, prompt)
            print("input:  ",  prompt)
            print("output: ", decoded)
            print()    
            
    def on_train_end(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        if model is None:
            return
        model.eval()
        print("="*20, f"ON TRAIN END", "="*20)        
        for prompt in self.prompts:
            decoded = self._one_answer(model, prompt)
            print("input:  ",  prompt)
            print("output: ", decoded)
            print()    


In [ ]:
# обучение модели

split_dataset = dataset.train_test_split(test_size=0.01)
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=MAX_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,  # 8 × 12 = 96
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=1 if DEVICE=='cpu' else 50,
    logging_steps=1 if DEVICE=='cpu' else 50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=hf_tokenizer,
    callbacks=[
        PromptCallback(
            tokenizer=hf_tokenizer,
            prompts=TEST_PROMPTS
        )
    ]
)
train_output = trainer.train()

In [ ]:
gc_collect()

## Post-train SFT

### Подготовка модели QWEN

In [4]:
tokenizer_qwen = AutoTokenizer.from_pretrained(
    MODEL_NAME, use_fast=True
)

model_qwen = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
)

### Ответы сырой модели

In [ ]:
def check_outputs(model, tokenizer, questtions):
    def _generate_answer(question):
        mess = [
            {"role": "user",
            "content": question}
            ]
        inputs = tokenizer.apply_chat_template(
            mess,
            return_tensors="pt",
            pad_token_id=tokenizer.pad_token_id
        ).to(model_qwen.device)

        output = model_qwen.generate(
            inputs,
            max_new_tokens=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
        return tokenizer.decode(output[0], skip_special_tokens=True)



    for idx, q in enumerate(questtions):
        print(f"Model Input {idx+1}:")
        print(q)
        print(f"Model Output {idx+1}:")
        print(_generate_answer(q))
        print("="*40)

check_outputs(model_qwen, tokenizer_qwen, QUESTIONS[:3])        

### Подготовка датасета

In [5]:
ds = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
len(ds)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b95bc971-81e8-4637-8193-4950bb7726f1)')' thrown while requesting HEAD https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru/resolve/05321f66bb70be21bd1ea2cb8fe3ca18693a688a/dataset_infos.json
Retrying in 1s [Retry 1/5].


51760

In [6]:
def example_to_message(example):
    return {
        "messages": [
            {'role': 'user', 'content': example['instruction']},
            {'role': 'assistant', 'content': example['output']}
        ]
    }

ds_chat = ds.map(
    example_to_message,
    batched=False,
    remove_columns=['input', 'instruction', 'output', ],
).map(
    lambda x: {
        'text': tokenizer_qwen.apply_chat_template(x['messages'],
         tokenize=False)
         },
        remove_columns=['messages'],
    )
ds_chat[0]    

{'text': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nДайте три совета, как оставаться здоровым.<|im_end|>\n<|im_start|>assistant\n1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здо

### Обучение модели QWEN

In [ ]:
split_ds_chat = ds_chat.train_test_split(test_size=0.01)
train_dataset = split_ds_chat['train']
val_dataset = split_ds_chat['test']


training_args_qwen = TrainingArguments(
    output_dir="./checkpoints_qwen",
    num_train_epochs=MAX_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE // 3,
    per_device_eval_batch_size=BATCH_SIZE // 3,
    gradient_accumulation_steps=8 * 3,
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=1 if DEVICE=='cpu' else 50,
    logging_steps=1 if DEVICE=='cpu' else 50,
    fp16=True,
    report_to="none",
)

trainer = SFTTrainer(
    model_qwen,
    args=training_args_qwen,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer_qwen,
    dataset_text_field = "text",    

)

train_output = trainer.train()

Map:   0%|          | 0/51242 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.59 GiB. GPU 0 has a total capacity of 14.58 GiB of which 639.62 MiB is free. Including non-PyTorch memory, this process has 13.95 GiB memory in use. Of the allocated memory 13.16 GiB is allocated by PyTorch, and 685.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
del model_qwen
gc_collect()